In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import CNN_Clasificator, LSTM__Clasificator
from get_embedded_data import get_data_glove_CNN, get_data_glove_LSTM, get_data_tokenizer_MLP, split_data, MAPPING

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Python311\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Python311\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warn

In [2]:
device = torch.device("cuda")
device

device(type='cuda')

In [3]:
# max_len_sentence = 0
# pos = 0
# for i, sentence in enumerate(list_of_words):
#     max_len_sentence, pos = (len(sentence), i) if len(sentence)>max_len_sentence else (max_len_sentence, pos)
# max_len_sentence, pos

In [4]:
# import nltk
# from nltk.tokenize import word_tokenize
# from gensim.models import Word2Vec
# # nltk.download('punkt')

# import gensim.downloader as api
# corpus = api.load('text8')

# with open('data_set.csv', 'r', encoding='utf-8') as dh:
#     list_of_words = []
#     list_of_targets = []
#     for i, line in enumerate(dh):
#         if i > 0:
#             line = line.strip()
#             line = line.split('@')
#             line[-1] = word_tokenize(line[-1].lower())
#             list_of_words.append(line[-1])
#             list_of_targets.append(line[1])
#     dh.close()

# emb_dim = 100

# model = Word2Vec(corpus, min_count=1)
# model.build_vocab(list_of_words, update=True)
# model.train(list_of_words, total_examples=model.corpus_count, epochs=20)
# model.save('word2vec_100d')

# # model = Word2Vec.load('word2vec/word2vec_100d')

# # Access embeddings
# word_embeddings = model.wv
# print(model.wv.most_similar("folly"))
# print(word_embeddings['superstitious'])


In [5]:
# word2vec_embeddings = []
# for sentence in list_of_words:
#     emb_sentence = np.empty((100,0))
#     for word in sentence:
#         emb_sentence = np.hstack((emb_sentence, np.reshape(model.wv[word], (100, 1))))
#     word2vec_embeddings.append(torch.from_numpy(emb_sentence))

In [6]:
# import torchtext
# import torchtext.vocab

# glove = torchtext.vocab.GloVe(name="6B", dim=100)

In [7]:
# list_of_inc = []
# for i, sen in enumerate(list_of_words):
#     counter = 0
#     for j, wor in enumerate(sen):
#         if wor not in glove:
#             counter += 1
#     if counter >= 1: list_of_inc.append((i, counter))
# print(len(list_of_inc))
# list_of_inc

In [8]:
# abc = []
# for i, sen in enumerate(list_of_words):
#     for j, wor in enumerate(sen):
#             if wor not in glove: abc.append((list_of_targets[i],i, j, wor))

In [9]:
# miss_by_aut=[]
# i = 0
# inc = 0
# prev_aut = list_of_inc[0][0]
# for aut , _, _ in list_of_inc:
#     if aut == prev_aut:
#         inc += 1
#     else:
#         miss_by_aut.append([prev_aut, inc])
#         prev_aut = aut
#         i+=1
#         inc = 0
# miss_by_aut.append([aut, inc])
# miss_by_aut

In [10]:
# import nltk

In [11]:
# nltk.download()

In [12]:
X_train, X_test, y_train, y_test = split_data("data_set.csv", "author", "quote", test_size=0.2, separator="@", mapping=MAPPING, labels_to_delete=[])

['aristotle' 'freud' 'hegel' 'kant' 'nietzsche' 'plato' 'sartre'
 'schopenhauer' 'spinoza']


In [13]:
# dataloader = get_data_BERT_MLP(20, device)
train_dataloader = get_data_glove_CNN(200, X_train, y_train)
test_dataloader = get_data_glove_CNN(200, X_test, y_test)

In [14]:
network = CNN_Clasificator().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters())

In [15]:
max_epoch = 30
network.train()
for epoch in range(max_epoch):

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = network(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        total += labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))
    running_loss = 0.0

print('Finished Training')

correct = 0
total = 0
for i, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = network(inputs)
    total += labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    correct += (predicted == labels).sum().item()

print(f"Test acc: {100 * correct / total}")

c:\Python311\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Python311\Lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


[1/30] loss: 0.010 accuracy: 15
[2/30] loss: 0.009 accuracy: 21
[3/30] loss: 0.009 accuracy: 20
[4/30] loss: 0.009 accuracy: 22
[5/30] loss: 0.009 accuracy: 24
[6/30] loss: 0.009 accuracy: 26
[7/30] loss: 0.009 accuracy: 27
[8/30] loss: 0.008 accuracy: 28
[9/30] loss: 0.008 accuracy: 31
[10/30] loss: 0.008 accuracy: 36
[11/30] loss: 0.008 accuracy: 38
[12/30] loss: 0.007 accuracy: 42
[13/30] loss: 0.007 accuracy: 47
[14/30] loss: 0.007 accuracy: 47
[15/30] loss: 0.006 accuracy: 52
[16/30] loss: 0.006 accuracy: 57
[17/30] loss: 0.005 accuracy: 60
[18/30] loss: 0.005 accuracy: 64
[19/30] loss: 0.005 accuracy: 67
[20/30] loss: 0.004 accuracy: 72
[21/30] loss: 0.004 accuracy: 76
[22/30] loss: 0.003 accuracy: 78
[23/30] loss: 0.003 accuracy: 80
[24/30] loss: 0.003 accuracy: 83
[25/30] loss: 0.003 accuracy: 84
[26/30] loss: 0.002 accuracy: 87
[27/30] loss: 0.002 accuracy: 88
[28/30] loss: 0.002 accuracy: 91
[29/30] loss: 0.002 accuracy: 93
[30/30] loss: 0.001 accuracy: 93
Finished Training
T

In [38]:
train_dataloader = get_data_glove_LSTM(200, X_train, y_train)
test_dataloader = get_data_glove_LSTM(1, X_test, y_test)

In [17]:
network = LSTM__Clasificator(80).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters())

In [19]:
max_epoch = 30
network.train()
for epoch in range(max_epoch):

    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels, x_len = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        network.prep(200)
        outputs = network(inputs)

        new_preds = torch.empty((0,9)).to(device)
        for i in range(len(outputs)):
            new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))

        loss = criterion(new_preds, labels)
        loss.backward()
        optimizer.step()


        total += labels.size(0)
        _, predicted = torch.max(new_preds.data, 1)
        correct += (predicted == labels).sum().item()

        running_loss += loss.item()

    print('[%d/%d] loss: %.3f accuracy: %d' %
          (epoch+1, max_epoch, running_loss / 2000, 100 * correct / total))
    running_loss = 0.0

c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\g

[1/30] loss: 0.007 accuracy: 46


c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\g

[2/30] loss: 0.007 accuracy: 49


c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens
c:\Users\bgale\Desktop\NLP\get_embedded_data.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(xx_pad, dtype=torch.double), torch.tensor(yy), x_lens


[3/30] loss: 0.007 accuracy: 48
[4/30] loss: 0.007 accuracy: 50
[5/30] loss: 0.006 accuracy: 51
[6/30] loss: 0.006 accuracy: 50
[7/30] loss: 0.006 accuracy: 52
[8/30] loss: 0.006 accuracy: 54
[9/30] loss: 0.006 accuracy: 56
[10/30] loss: 0.006 accuracy: 56
[11/30] loss: 0.006 accuracy: 57
[12/30] loss: 0.006 accuracy: 55
[13/30] loss: 0.006 accuracy: 59
[14/30] loss: 0.005 accuracy: 59
[15/30] loss: 0.006 accuracy: 56
[16/30] loss: 0.006 accuracy: 53
[17/30] loss: 0.006 accuracy: 56
[18/30] loss: 0.006 accuracy: 55
[19/30] loss: 0.006 accuracy: 57
[20/30] loss: 0.005 accuracy: 60
[21/30] loss: 0.005 accuracy: 62
[22/30] loss: 0.005 accuracy: 65
[23/30] loss: 0.005 accuracy: 66
[24/30] loss: 0.004 accuracy: 68
[25/30] loss: 0.004 accuracy: 68
[26/30] loss: 0.004 accuracy: 70
[27/30] loss: 0.004 accuracy: 70
[28/30] loss: 0.004 accuracy: 72
[29/30] loss: 0.004 accuracy: 71
[30/30] loss: 0.004 accuracy: 67


In [39]:
correct = 0
total = 0
for i, data in enumerate(test_dataloader, 0):
    inputs, labels, x_len = data
    inputs, labels = inputs.to(device), labels.to(device)
    network.prep(1)
    outputs = network(inputs)
    new_preds = torch.empty((0,9)).to(device)
    for i in range(len(outputs)):
        new_preds = torch.vstack((new_preds, outputs[i][x_len[i]-1]))
    total += labels.size(0)
    _, predicted = torch.max(new_preds.data, 1)
    correct += (predicted == labels).sum().item()

print(f"Test acc: {100 * correct / total}")

Test acc: 31.17408906882591
